In [38]:
import garminconnect
import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

email = os.getenv("email")
password = os.getenv("garminpassword")\

garmin = garminconnect.Garmin(email, password)
garmin.login()
garmin.display_name

'5f3dc0bc-4916-4755-894b-cca99162502b'

In [39]:
import json
date = '2025-06-12'
def getdata(date):
    return garmin.get_sleep_data(date)

data = getdata(date)

In [40]:
def safe_dataframe(data, key):
    """
    Safely extracts a nested value from a dictionary and returns a DataFrame.
    Handles missing keys, None values, scalars, or poorly structured dicts.
    """
    nested = data.get(key)

    if not nested:
        return pd.DataFrame()

    # If a dict but all values are scalars (not dicts/lists), skip it
    if isinstance(nested, dict):
        if all(not isinstance(v, (dict, list)) for v in nested.values()):
            return pd.DataFrame()
        return pd.DataFrame([nested])  
    
    # If list of struct objects or list
    if isinstance(nested, list):
        return pd.DataFrame(nested)

    # return empty 
    return pd.DataFrame()

In [41]:
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DATABASE_URL)

try:
    with engine.connect() as conn:
        result = result = conn.execute(text("SELECT version();"))
        for row in result:
            print("Connected to:", row[0])
except Exception as e:
    print(" Connection failed:", e)

Connected to: PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


In [42]:
def getgeneral(data):
        return safe_dataframe(data, 'dailySleepDTO')

df_general = getgeneral(data)
print(df_general.head())

              id  userProfilePK calendarDate  sleepTimeSeconds  \
0  1749687960000       80897268   2025-06-12             24660   

   napTimeSeconds  sleepWindowConfirmed sleepWindowConfirmationType  \
0               0                  True    enhanced_confirmed_final   

   sleepStartTimestampGMT  sleepEndTimestampGMT  sleepStartTimestampLocal  \
0           1749687960000         1749713640000             1749691560000   

   ...  lowestRespirationValue highestRespirationValue awakeCount  \
0  ...                    10.0                    20.0          1   

  avgSleepStress ageGroup  sleepScoreFeedback  sleepScoreInsight  \
0           19.0    ADULT       POSITIVE_DEEP               NONE   

   sleepScorePersonalizedInsight  \
0                  NOT_AVAILABLE   

                                         sleepScores  sleepVersion  
0  {'totalDuration': {'qualifierKey': 'FAIR', 'op...             2  

[1 rows x 38 columns]


In [43]:
def getsleepHR(data):
    
    df = safe_dataframe(data, 'sleepHeartRate')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    
    return df

df_sleephr = getsleepHR(data)
print(df_sleephr.head())

   value            startGMT
0     82 2025-06-12 00:26:00
1     82 2025-06-12 00:28:00
2     79 2025-06-12 00:30:00
3     76 2025-06-12 00:32:00
4     78 2025-06-12 00:34:00


In [44]:
def getsleep_wakeHR(date):
    if df_sleephr.empty:
        return {'Date':date,'sleepHR': None, 'wakeHR': None}

    df_sorted = df_sleephr.sort_values('startGMT').reset_index(drop=True)

    sleep_hr = df_sorted.loc[0, 'value']
    wake_hr = df_sorted.loc[len(df_sorted) - 1, 'value']

    row = pd.DataFrame([{
        'date': date,
        'sleep_hr': sleep_hr,
        'wake_hr': wake_hr
    }])

    return row



print(getsleep_wakeHR(date))

         date  sleep_hr  wake_hr
0  2025-06-12        82       64


In [45]:
current_date = datetime.date.today()
end_date = datetime.date(2024, 6, 19)

df_sleepwakehr = pd.DataFrame()

while current_date >= end_date:
    data = getdata(current_date)
    df_general = getgeneral(data)
    df_sleephr = getsleepHR(data)
    df_sleep_wake = getsleep_wakeHR(current_date)
    print(df_sleep_wake)
    if not df_sleephr.empty:
        df_sleep_wake = getsleep_wakeHR(current_date)
        df_sleepwakehr = pd.concat([df_sleepwakehr, df_sleep_wake], ignore_index=True)

    current_date -= datetime.timedelta(days=1)

df_sleepwakehr['date'] = pd.to_datetime(df_sleepwakehr['date']).dt.date
df_sleepwakehr.head()


         date  sleep_hr  wake_hr
0  2025-07-14        56       50
         date  sleep_hr  wake_hr
0  2025-07-13        64       47
         date  sleep_hr  wake_hr
0  2025-07-12        61       45
         date  sleep_hr  wake_hr
0  2025-07-11        55       50
         date  sleep_hr  wake_hr
0  2025-07-10        56       45
         date  sleep_hr  wake_hr
0  2025-07-09        61       56
         date  sleep_hr  wake_hr
0  2025-07-08        60       45
         date  sleep_hr  wake_hr
0  2025-07-07      70.0     58.0
         date  sleep_hr  wake_hr
0  2025-07-06        60       52
         date  sleep_hr  wake_hr
0  2025-07-05        69       48
         date  sleep_hr  wake_hr
0  2025-07-04        72       46
{'Date': datetime.date(2025, 7, 3), 'sleepHR': None, 'wakeHR': None}
         date  sleep_hr  wake_hr
0  2025-07-02        54       49
         date  sleep_hr  wake_hr
0  2025-07-01        65       50
         date  sleep_hr  wake_hr
0  2025-06-30        55       46
       

,date,sleep_hr,wake_hr
0,2025-07-14,56.0,50.0
1,2025-07-13,64.0,47.0
2,2025-07-12,61.0,45.0
3,2025-07-11,55.0,50.0
4,2025-07-10,56.0,45.0


In [51]:
update_query = text("""
    UPDATE simplesleepdata
    SET 
        "sleepHR" = :sleep_hr,
        "wakeHR" = :wake_hr
    WHERE date = :date
""")

with engine.connect() as conn:
    for _, row in df_sleepwakehr.iterrows():
        conn.execute(update_query, {
            "sleep_hr": row.get('sleep_hr', None),
            "wake_hr": row.get('wake_hr', None),
            "date": row['date'].strftime("%Y-%m-%d")
        })
    conn.commit()